In [21]:
import pandas as pd
import numpy as np

In [22]:
reviews = pd.read_csv("fr_reviews.csv")

In [23]:
reviews

,Unnamed: 0,text,sentiment
0,0,Éoliennes Belle-Rivière: fin de la période d'i...,=
1,1,La stratégie nationale de gestion des risques ...,+
2,2,“Nous cherchons à changer l’économie mondiale ...,=
3,3,"""La technologie, clé contre le changement clim...",+
4,4,@JMoncomble Probablement une histoire de dével...,-
...,...,...,...
262414,259035,Les quelques recettes de base sont bien expliq...,=
262415,259036,Ma fille a command&eacute; ce livre apr&egrave...,+
262416,259037,Bill and Melinda recommended this book to me a...,+
262417,259038,Un classique a avoir dans sa bibliotheque. Tou...,+


In [24]:
reviews = reviews[:50000].copy()

In [25]:
reviews

,Unnamed: 0,text,sentiment
0,0,Éoliennes Belle-Rivière: fin de la période d'i...,=
1,1,La stratégie nationale de gestion des risques ...,+
2,2,“Nous cherchons à changer l’économie mondiale ...,=
3,3,"""La technologie, clé contre le changement clim...",+
4,4,@JMoncomble Probablement une histoire de dével...,-
...,...,...,...
49995,46616,"un enregistrement DECCA d'une grande qualité ,...",+
49996,46617,"Haletant, déroutant, ce film historique est à ...",+
49997,46618,Une BO de bonne facture mais clairement sur&ea...,=
49998,46619,Krakauer mène un excellent travail d'investiga...,+


In [26]:
reviews.drop(labels=['Unnamed: 0'], axis=1,inplace=True)

In [27]:
reviews

,text,sentiment
0,Éoliennes Belle-Rivière: fin de la période d'i...,=
1,La stratégie nationale de gestion des risques ...,+
2,“Nous cherchons à changer l’économie mondiale ...,=
3,"""La technologie, clé contre le changement clim...",+
4,@JMoncomble Probablement une histoire de dével...,-
...,...,...
49995,"un enregistrement DECCA d'une grande qualité ,...",+
49996,"Haletant, déroutant, ce film historique est à ...",+
49997,Une BO de bonne facture mais clairement sur&ea...,=
49998,Krakauer mène un excellent travail d'investiga...,+


In [28]:
from sklearn.preprocessing import LabelEncoder

In [29]:
le = LabelEncoder()
le.fit(reviews["sentiment"])
reviews["encoded_target"] = le.transform(reviews["sentiment"])

In [30]:
reviews

,text,sentiment,encoded_target
0,Éoliennes Belle-Rivière: fin de la période d'i...,=,2
1,La stratégie nationale de gestion des risques ...,+,0
2,“Nous cherchons à changer l’économie mondiale ...,=,2
3,"""La technologie, clé contre le changement clim...",+,0
4,@JMoncomble Probablement une histoire de dével...,-,1
...,...,...,...
49995,"un enregistrement DECCA d'une grande qualité ,...",+,0
49996,"Haletant, déroutant, ce film historique est à ...",+,0
49997,Une BO de bonne facture mais clairement sur&ea...,=,2
49998,Krakauer mène un excellent travail d'investiga...,+,0


In [31]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

In [32]:
def get_vectorized_text():
  vectorizer = TfidfVectorizer()
  X = vectorizer.fit_transform(reviews["text"])
  return X

In [33]:
X = get_vectorized_text()

In [34]:
from sklearn.model_selection import train_test_split  

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X,reviews["encoded_target"], test_size=.2, random_state=42,shuffle=True)

In [36]:
X_train.shape,y_train.shape

((40000, 91571), (40000,))

In [37]:
x_train_chunks = [X_train[i:i+16000] for i in range(0, X_train.shape[0], 16000) ]
y_train_chunks = [y_train[i:i+16000] for i in range(0, y_train.shape[0], 16000) ]

In [38]:
x_train_chunks

[<16000x91571 sparse matrix of type '<class 'numpy.float64'>'
 	with 692718 stored elements in Compressed Sparse Row format>,
 <16000x91571 sparse matrix of type '<class 'numpy.float64'>'
 	with 686351 stored elements in Compressed Sparse Row format>,
 <8000x91571 sparse matrix of type '<class 'numpy.float64'>'
 	with 336697 stored elements in Compressed Sparse Row format>]

In [39]:
clf = OneVsRestClassifier(MultinomialNB())

In [40]:
for i,j in zip(x_train_chunks,y_train_chunks):
    clf.partial_fit(i.toarray(),j, classes = np.unique(y_train))

In [50]:
clf.score(X_test,y_test)

0.8006

In [51]:
import pickle

In [52]:
pickle.dump(clf,open("sentiment_model.h5","wb"))

In [53]:
model = pickle.load(open("sentiment_model.h5","rb"))